In [2]:
from datetime import datetime
import csv
import pandas as pd
import os

In [12]:
csv_file = '/content/drive/MyDrive/Project/Smart Bill Analyzer/isp.csv'
if not os.path.isfile(csv_file):
    # If the file doesn't exist, create it with a header
    df = pd.DataFrame(columns=['image_path', 'ISP', 'date', 'amount'])
    df.to_csv(csv_file, index=False)
else:
    # If the file exists, append the data to the DataFrame
    df = pd.read_csv(csv_file)

In [4]:
def convert_date(date_str):
    try:
        # Attempt to parse the date in "month/day/year" format
        date_obj = datetime.strptime(date_str, "%d/%m/%Y")
        # Convert it to "year-month-day" format
        return date_obj.strftime("%Y-%m-%d")
    except ValueError:
        # If the date is already in "year-month-day" format or is not a valid date, return it as is
        return date_str

In [5]:
folder_path = '/content/drive/MyDrive/Project/Smart Bill Analyzer/ISP'

# Get a list of all files in the folder
file_list = os.listdir(folder_path)


# Filter the list to keep only image files (you can add more extensions if needed)
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
image_files = [f for f in file_list if any(f.lower().endswith(ext) for ext in image_extensions)]


In [6]:
pip install azure-ai-formrecognizer==3.3.0

In [7]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

"""
Remember to remove the key from your code when you're done, and never post it publicly. For production, use
secure methods to store and access your credentials. For more information, see
https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-security?tabs=command-line%2Ccsharp#environment-variables-and-application-configuration
"""

endpoint = "https://thilakna-doc-intelligence-instance.cognitiveservices.azure.com/"
key = "29579bf5af1f4559bb8228d643e79d7b"
model_id = "ISP-M1"

# Initialize the DocumentAnalysisClient
document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

In [ ]:
# Loop through the image files and open each one
for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(folder_path, image_file)
    # if image_path in df['image_path'].tolist():
    #     print(f"Skipping image {image_file} as it's already in the DataFrame.")
    #     continue


    try:
      # Analyze the document using the custom model
      with open(image_path, "rb") as document:
        poller = document_analysis_client.begin_analyze_document(model_id, document)
      result = poller.result()
            # Process the analysis results as before
      for idx, document in enumerate(result.documents):
          data = []
          for name, field in document.fields.items():
              field_value = field.value if field.value else field.content
              data.append(field_value)

          # Append the data to the existing CSV file
          data_dict = {
              "image_path": [image_path],
              "date":[data[2]],  # Adjust the index to match the data
              "amount": [data[0]],  # Adjust the index to match the data
              "isp": [data[1]],  # Adjust the index to match the data
          }
          df = pd.DataFrame(data_dict)
          df.to_csv('/content/drive/MyDrive/Project/Smart Bill Analyzer/isp.csv', mode='a', header=False, index=False)
    except Exception as e:
      print(e)



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('../files/isp_data_2.csv')
df